In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.datasets import load_boston
from sklearn.metrics import r2_score, mean_squared_log_error, mean_squared_error,mean_absolute_error
import datetime as dt
import pickle
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import preparing_data as F

In [2]:
data=pd.read_pickle("./dataframe/df_20210912_133401_ONLY_RISKY_EVENTS-6-withFE.pkl")
data.reset_index(inplace=True)
data.drop(['index'], inplace=True, axis=1)
print(data.shape)
data.head()

(272, 86)


,__time_to_tca,MISS_DISTANCE,RELATIVE_SPEED,RELATIVE_POSITION_R,RELATIVE_POSITION_T,RELATIVE_POSITION_N,RELATIVE_VELOCITY_R,RELATIVE_VELOCITY_T,RELATIVE_VELOCITY_N,COLLISSION_PROBABILITY,...,PC_mavg_1,PC_trend_1,PC_trend_3,PC_gradient_1,PC_gradient_3,MD_mavg_1,MD_trend_1,MD_trend_3,MD_gradient_1,MD_gradient_3
0,0.832257,2244.0,11172.0,87.9,-1474.9,1689.9,-201.4,-8424.3,-7336.0,-3.418392,...,-3.475175,0.132439,0.590914,0.354472,0.579354,2181.333333,47.0,17.0,125.795356,16.667421
1,0.580855,966.0,212.0,191.0,-944.8,-68.9,33.8,-8.4,209.3,-3.711974,...,-3.981494,0.310256,0.575376,0.747481,0.536846,1131.000000,-201.0,-326.0,-484.256505,-304.169574
2,0.189825,1102.0,212.0,193.3,-1083.1,-75.0,33.9,-8.4,209.3,-3.913996,...,-3.882734,-0.202022,0.296282,-0.516640,0.270366,1078.333333,136.0,-158.0,347.799165,-144.179478
3,0.994637,456.0,11707.0,-22.1,288.8,353.3,-10.2,-9077.7,7392.6,-2.055073,...,-2.841298,1.280567,1.312874,3.634346,1.445698,709.666667,-577.0,124.0,-1637.570088,136.545083
4,0.759012,629.0,11707.0,-18.7,395.3,489.9,-10.3,-9077.7,7392.6,-2.649558,...,-2.680091,-0.594485,0.483623,-2.523011,0.486607,706.000000,173.0,-11.0,734.217036,-11.067877


In [3]:
data.shape

(272, 86)

In [4]:
train, test = train_test_split(data, test_size=0.30, random_state=42)

In [5]:
print("Train dataframe dimension {} x {}".format(train.shape[0],train.shape[1]))
print("Test dataframe dimension {} x {}".format(test.shape[0],test.shape[1]))

Train dataframe dimension 190 x 86
Test dataframe dimension 82 x 86


In [6]:
Y_train = train["COLLISSION_PROBABILITY"]
X_train= train.drop(["COLLISSION_PROBABILITY"], axis=1)
Y_test = test["COLLISSION_PROBABILITY"]
X_test= test.drop(["COLLISSION_PROBABILITY"], axis=1)


In [7]:
X = X_train
y = Y_train

In [8]:
# improving function
def bayesian_opt_lgbm(X, y, init_iter=3, n_iters=7, random_state=11, seed = 101, num_iterations = 100,evalm="lgb_r2"):
      dtrain = lgb.Dataset(data=X, label=y)
      #Metric evaluation functions
      def lgb_r2(preds, dtrain):                #R2
            labels = dtrain.get_label()
            return 'metric', r2_score(labels, preds), True
      def lgb_rmse(preds, dtrain):      #RMSE
            labels = dtrain.get_label()
            return 'metric', mean_squared_error(labels, preds,squared=False), True
      def lgb_mae(preds, dtrain):     #MAE
            labels = dtrain.get_label()
            return 'metric', mean_absolute_error(labels, preds), True
      def lgb_adjusted_r2(preds, dtrain):       #ADJUSTED R2
            labels = dtrain.get_label()
            n=dtrain.num_data()
            k=dtrain.num_feature()
            return 'metric', ((1-r2_score(labels, preds))*(n-1))/(n-k-1), True


      metrics_dict= {   "lgb_r2" : lgb_r2,
                        "lgb_rmse":lgb_rmse,
                        "lgb_mae":lgb_mae,
                        "lgb_adjusted_r2": lgb_adjusted_r2
                        }
      # Select metric
      metric=str(evalm)
      metric_feval=metrics_dict.get(str(evalm))

      # Objective Function
      def hyp_lgbm(num_leaves, feature_fraction, learning_rate, bagging_fraction, max_depth, min_split_gain, min_child_weight):
              params = {      'application':'regression',
                              'num_iterations': num_iterations,
                              'early_stopping_round':50,
                              'verbose':-1,
                              'metric':metric} # Default parameters
              params["num_leaves"] = int(round(num_leaves))
              params["learning_rate"] = learning_rate
              params['feature_fraction'] = max(min(feature_fraction, 1), 0)
              params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
              params['max_depth'] = int(round(max_depth))
              params['min_split_gain'] = min_split_gain
              params['min_child_weight'] = min_child_weight
              cv_results = lgb.cv(params, dtrain, nfold=5, seed=seed,categorical_feature=[], stratified=False,
                                  verbose_eval =None, feval=metric_feval)
              #print(cv_results)
              return np.max(cv_results['metric-mean'])
    
              # Domain space-- Range of hyperparameters 
      pds = {     'num_leaves': (60, 120),
                  'feature_fraction': (0.1, 0.9),
                  'bagging_fraction': (0.7, 1),
                  'max_depth': (7, 15),
                  'learning_rate':(0.001,0.05), 
                  'min_split_gain': (0.001, 0.1),
                  'min_child_weight': (10, 35)
                  }
      # Surrogate model
      optimizer = BayesianOptimization(hyp_lgbm, pds, random_state=random_state)
                                          
      # Optimize
      optimizer.maximize(init_points=init_iter, n_iter=n_iters)

      # Output dictionary
      output_dict=optimizer.max['params']
      output_dict["num_iterations"]=num_iterations
      output_dict["n_estimators"]=n_iters

      # Save dictionary to file
      filename="./opt_parameters_bo/param_{}_{}.pkl".format(dt.datetime.now().strftime("%Y%m%d_%H%M%S"),metric)
      a_file = open(filename, "wb")
      pickle.dump(output_dict, a_file)
      a_file.close()

      return optimizer,output_dict

In [9]:
def compare_true_vs_prediction(df_true,df_pred):
    aux_y=pd.DataFrame(df_true)
    aux_y.reset_index(inplace=True)
    aux_y.drop(['index'], inplace=True, axis=1)
    aux_y_pred=pd.DataFrame(df_pred)
    aux_y_pred.reset_index(inplace=True)
    aux_y_pred.drop(['index'], inplace=True, axis=1)
    frames=[aux_y,aux_y_pred]
    result=pd.concat(frames,axis=1)
    result.columns=["y_true","y_predicted"]
    result["y_true_10"]=10**result.y_true
    result["y_predicted_10"]=10**result.y_predicted
    result[result["y_true_10"]>0.00001]
    result[result["y_true_10"]>0.0001]
    return result


In [10]:
def create_and_validate_model(X, y, 
                            init_iter=5, n_iters=500, random_state=77, seed = 101,num_iterations=300,
                            evalm="lgb_r2",hp_metric="regression_L2"):
    bayesian=bayesian_opt_lgbm(X, y, init_iter, n_iters, random_state, seed,num_iterations,evalm)
    opt_parameters=bayesian[1]
    print("------------------------ OPTIMAL PARAMETERS ------------------------")
    print(opt_parameters)
    print("-------------------------------------------------------------------")
    
    # LOAD OPTIMAL PARAMETERS FOR FURTHER COMPUTATION
    hyper_params = {
                    'task': 'train',
                    'boosting_type': 'gbdt',
                    'objective': 'regression',
                    'metric': str(hp_metric),
                    'learning_rate': opt_parameters.get("learning_rate"),
                    'feature_fraction': opt_parameters.get("feature_fraction"),
                    'bagging_fraction': opt_parameters.get("bagging_fraction"),
                    'verbose': -1,
                    "max_depth": int(round(opt_parameters.get("max_depth"))),
                    "num_leaves": int(round(opt_parameters.get("num_leaves"))),  
                    'min_split_gain' : opt_parameters.get("min_split_gain"),
                    "num_iterations": opt_parameters.get("num_iterations"),
                    "n_estimators": opt_parameters.get("n_estimators"),
                    'min_child_weight' : opt_parameters.get("min_child_weight")
                    }
    # TRAIN MODEL WITH OPTIMAL PARAMETERS
    lgbm_train = lgb.Dataset(X, label=y)
    gbm = lgb.train(params=hyper_params,train_set=lgbm_train)

    # TEST MODEL WITH TESTING SUBPART OF DATASET
    Y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

    # REGRESION MODEL METRICS
    print('The r2 of prediction is:', r2_score(Y_test, Y_pred))
    print('The MSE of prediction is:', mean_squared_error(Y_test, Y_pred, squared=True))
    print('The RMSE of prediction is:', mean_squared_error(Y_test, Y_pred, squared=False))
    print('The MAE of prediction is:', mean_absolute_error(Y_test, Y_pred))

    # COMPARE TEST VALUES VS PREDICTED VALUES
    df_results=compare_true_vs_prediction(df_true=Y_test,df_pred=Y_pred)

    # WRITE TO A FILE
    
    outF = open("./validation-results/r_evalm-{}_hp_metric-_{}_{}.txt".format(str(evalm),str(hp_metric),dt.datetime.now().strftime("%Y%m%d-%H%M%S")), "w+")
    # write line to output file
    outF.write("------------------------- MODEL HYPER-PARAMETERS ------------------------- \n")
    outF.write(str(hyper_params))
    outF.write("\n")
    outF.write("\n")
    outF.write("------------------------ REGRESSION MODEL METRICS ------------------------ \n")
    outF.write(str('The r2 of prediction is: ') + str(r2_score(Y_test, Y_pred))+str("\n"))
    outF.write(str('The MSE of prediction is: ') + str(mean_squared_error(Y_test, Y_pred, squared=True))+str("\n"))
    outF.write(str('The RMSE of prediction is: ')+ str(mean_squared_error(Y_test, Y_pred, squared=False))+str("\n"))
    outF.write(str('The MAE of prediction is: ')+ str(mean_absolute_error(Y_test, Y_pred))+str("\n"))
    outF.write("\n")
    outF.write("\n")
    outF.write("-------------------------- ADDITIONAL COMMENTS -------------------------- \n")
    outF.write(str("This model was created and validated at {}".format(dt.datetime.fromtimestamp(dt.datetime.timestamp(dt.datetime.now())))))
    outF.close()

    return gbm, df_results

In [16]:
def load_and_validate_model(name,X,y,hp_metric="regression_L2"):
    filename=str("./opt_parameters_bo/{}.pkl".format(name))
    a_file = open(filename,"rb")
    opt_parameters = pickle.load(a_file)
    print("------------------------ OPTIMAL PARAMETERS ------------------------")
    print(opt_parameters)
    print("-------------------------------------------------------------------")
    
    # LOAD OPTIMAL PARAMETERS FOR FURTHER COMPUTATION
    hyper_params = {
                    'task': 'train',
                    'boosting_type': 'gbdt',
                    'objective': 'regression',
                    'metric': str(hp_metric),
                    'learning_rate': opt_parameters.get("learning_rate"),
                    'feature_fraction': opt_parameters.get("feature_fraction"),
                    'bagging_fraction': opt_parameters.get("bagging_fraction"),
                    'verbose': -1,
                    "max_depth": int(round(opt_parameters.get("max_depth"))),
                    "num_leaves": int(round(opt_parameters.get("num_leaves"))),  
                    'min_split_gain' : opt_parameters.get("min_split_gain"),
                    "num_iterations": opt_parameters.get("num_iterations"),
                    "n_estimators": opt_parameters.get("n_estimators"),
                    'min_child_weight' : opt_parameters.get("min_child_weight")
                    }
    # TRAIN MODEL WITH OPTIMAL PARAMETERS
    lgbm_train = lgb.Dataset(X, label=y)
    gbm = lgb.train(params=hyper_params,train_set=lgbm_train)

    # TEST MODEL WITH TESTING SUBPART OF DATASET
    Y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

    # REGRESION MODEL METRICS
    print('The r2 of prediction is:', r2_score(Y_test, Y_pred))
    print('The MSE of prediction is:', mean_squared_error(Y_test, Y_pred, squared=True))
    print('The RMSE of prediction is:', mean_squared_error(Y_test, Y_pred, squared=False))
    print('The MAE of prediction is:', mean_absolute_error(Y_test, Y_pred))

    # COMPARE TEST VALUES VS PREDICTED VALUES
    df_results=compare_true_vs_prediction(df_true=Y_test,df_pred=Y_pred)

    # WRITE TO A FILE
    
    outF = open("./validation-results/r_hp_metric-_{}_{}.txt".format(str(hp_metric),dt.datetime.now().strftime("%Y%m%d-%H%M%S")), "w+")
    # write line to output file
    outF.write("------------------------- MODEL HYPER-PARAMETERS ------------------------- \n")
    outF.write(str(hyper_params))
    outF.write("\n")
    outF.write("\n")
    outF.write("------------------------ REGRESSION MODEL METRICS ------------------------ \n")
    outF.write(str('The r2 of prediction is: ') + str(r2_score(Y_test, Y_pred))+str("\n"))
    outF.write(str('The MSE of prediction is: ') + str(mean_squared_error(Y_test, Y_pred, squared=True))+str("\n"))
    outF.write(str('The RMSE of prediction is: ')+ str(mean_squared_error(Y_test, Y_pred, squared=False))+str("\n"))
    outF.write(str('The MAE of prediction is: ')+ str(mean_absolute_error(Y_test, Y_pred))+str("\n"))
    outF.write("\n")
    outF.write("\n")
    outF.write("-------------------------- ADDITIONAL COMMENTS -------------------------- \n")
    outF.write(str("This model was created and validated at {}".format(dt.datetime.fromtimestamp(dt.datetime.timestamp(dt.datetime.now())))))
    outF.close()

    return gbm, df_results



In [11]:
create_and_validate_model(X, y,
                        init_iter=5, n_iters=1000, random_state=77, seed = 101,num_iterations=300)

|   iter    |  target   | baggin... | featur... | learni... | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.848    |  0.9757   |  0.6138   |  0.03793  |  8.115    |  12.18    |  0.07901  |  79.57    |
|  2        |  0.8      |  0.8623   |  0.2922   |  0.02773  |  10.2     |  27.88    |  0.08383  |  95.31    |
|  3        |  0.8222   |  0.7888   |  0.3248   |  0.03557  |  10.38    |  11.43    |  0.07496  |  87.14    |
|  4        |  0.5445   |  0.7527   |  0.1395   |  0.01533  |  7.534    |  28.78    |  0.007313 |  85.91    |
|  5        |  0.7843   |  0.8093   |  0.2216   |  0.02779  |  10.55    |  10.9     |  0.08247  |  76.4     |
|  6        |  0.8548   |  1.0      |  0.9      |  0.05     |  7.0      |  10.0     |  0.1      |  120.0    |
|  7        |  0.8325   |  1.0      |  0.9      |  0.05     |  15.0     |  35.0     |  0.001    |  120.0    |
|  8      

(<lightgbm.basic.Booster at 0x7f5aaf303c90>,
       y_true  y_predicted  y_true_10  y_predicted_10
 0  -4.558148    -4.622216   0.000028        0.000024
 1  -5.849858    -5.861389   0.000001        0.000001
 2  -4.882729    -5.137882   0.000013        0.000007
 3  -5.337053    -5.338532   0.000005        0.000005
 4  -5.325139    -5.367577   0.000005        0.000004
 ..       ...          ...        ...             ...
 77 -5.017593    -4.995105   0.000010        0.000010
 78 -5.578232    -5.511897   0.000003        0.000003
 79 -4.345823    -4.264230   0.000045        0.000054
 80 -5.901702    -5.714671   0.000001        0.000002
 81 -4.529737    -3.932187   0.000030        0.000117
 
 [82 rows x 4 columns])

In [12]:
# filename="./opt_parameters_bo/param_20210912_213528_lgb_r2.pkl"
# a_file = open(filename,"rb")
# output = pickle.load(a_file)
# opt_parameters=output
# output

In [13]:
create_and_validate_model(X, y,
                        init_iter=5, n_iters=1000, random_state=77, seed = 101,num_iterations=300,evalm="lgb_mae",hp_metric="mae")

|   iter    |  target   | baggin... | featur... | learni... | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.5654   |  0.9757   |  0.6138   |  0.03793  |  8.115    |  12.18    |  0.07901  |  79.57    |
|  2        |  0.5664   |  0.8623   |  0.2922   |  0.02773  |  10.2     |  27.88    |  0.08383  |  95.31    |
|  3        |  0.5643   |  0.7888   |  0.3248   |  0.03557  |  10.38    |  11.43    |  0.07496  |  87.14    |
|  4        |  0.5702   |  0.7527   |  0.1395   |  0.01533  |  7.534    |  28.78    |  0.007313 |  85.91    |
|  5        |  0.5685   |  0.8093   |  0.2216   |  0.02779  |  10.55    |  10.9     |  0.08247  |  76.4     |
|  6        |  0.5663   |  0.9346   |  0.3943   |  0.0391   |  14.38    |  33.97    |  0.04177  |  60.81    |
|  7        |  0.5654   |  0.9702   |  0.6933   |  0.04408  |  7.681    |  34.7     |  0.09668  |  119.8    |
|  8      

(<lightgbm.basic.Booster at 0x7f5ab0ed0190>,
       y_true  y_predicted  y_true_10  y_predicted_10
 0  -4.558148    -5.021883   0.000028        0.000010
 1  -5.849858    -5.352879   0.000001        0.000004
 2  -4.882729    -5.310708   0.000013        0.000005
 3  -5.337053    -5.276611   0.000005        0.000005
 4  -5.325139    -5.113177   0.000005        0.000008
 ..       ...          ...        ...             ...
 77 -5.017593    -5.260218   0.000010        0.000005
 78 -5.578232    -5.181327   0.000003        0.000007
 79 -4.345823    -4.876086   0.000045        0.000013
 80 -5.901702    -5.145369   0.000001        0.000007
 81 -4.529737    -4.790934   0.000030        0.000016
 
 [82 rows x 4 columns])

In [14]:
create_and_validate_model(X, y,
                        init_iter=5, n_iters=1000, random_state=77, seed = 101,num_iterations=300,evalm="lgb_rmse",hp_metric="rmse")

|   iter    |  target   | baggin... | featur... | learni... | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.7189   |  0.9757   |  0.6138   |  0.03793  |  8.115    |  12.18    |  0.07901  |  79.57    |
|  2        |  0.7195   |  0.8623   |  0.2922   |  0.02773  |  10.2     |  27.88    |  0.08383  |  95.31    |
|  3        |  0.7173   |  0.7888   |  0.3248   |  0.03557  |  10.38    |  11.43    |  0.07496  |  87.14    |
|  4        |  0.7238   |  0.7527   |  0.1395   |  0.01533  |  7.534    |  28.78    |  0.007313 |  85.91    |
|  5        |  0.7222   |  0.8093   |  0.2216   |  0.02779  |  10.55    |  10.9     |  0.08247  |  76.4     |
|  6        |  0.7196   |  0.9346   |  0.3943   |  0.0391   |  14.38    |  33.97    |  0.04177  |  60.81    |
|  7        |  0.7184   |  0.9702   |  0.6933   |  0.04408  |  7.681    |  34.7     |  0.09668  |  119.8    |
|  8      

(<lightgbm.basic.Booster at 0x7f5ab0e919d0>,
       y_true  y_predicted  y_true_10  y_predicted_10
 0  -4.558148    -5.069362   0.000028        0.000009
 1  -5.849858    -5.346316   0.000001        0.000005
 2  -4.882729    -5.326647   0.000013        0.000005
 3  -5.337053    -5.291529   0.000005        0.000005
 4  -5.325139    -5.109155   0.000005        0.000008
 ..       ...          ...        ...             ...
 77 -5.017593    -5.268990   0.000010        0.000005
 78 -5.578232    -5.168399   0.000003        0.000007
 79 -4.345823    -4.923433   0.000045        0.000012
 80 -5.901702    -5.096304   0.000001        0.000008
 81 -4.529737    -4.807279   0.000030        0.000016
 
 [82 rows x 4 columns])

In [15]:
create_and_validate_model(X, y,
                        init_iter=5, n_iters=1000, random_state=77, seed = 101,num_iterations=300,evalm="lgb_adjusted_r2")

|   iter    |  target   | baggin... | featur... | learni... | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------
|  1        | -0.1172   |  0.9757   |  0.6138   |  0.03793  |  8.115    |  12.18    |  0.07901  |  79.57    |
|  2        | -0.1541   |  0.8623   |  0.2922   |  0.02773  |  10.2     |  27.88    |  0.08383  |  95.31    |
|  3        | -0.137    |  0.7888   |  0.3248   |  0.03557  |  10.38    |  11.43    |  0.07496  |  87.14    |
|  4        | -0.3511   |  0.7527   |  0.1395   |  0.01533  |  7.534    |  28.78    |  0.007313 |  85.91    |
|  5        | -0.1663   |  0.8093   |  0.2216   |  0.02779  |  10.55    |  10.9     |  0.08247  |  76.4     |
|  6        | -0.1119   |  1.0      |  0.9      |  0.05     |  7.0      |  10.0     |  0.1      |  120.0    |
|  7        | -0.1291   |  1.0      |  0.9      |  0.05     |  15.0     |  35.0     |  0.001    |  120.0    |
|  8      

(<lightgbm.basic.Booster at 0x7f5ae073fa50>,
       y_true  y_predicted  y_true_10  y_predicted_10
 0  -4.558148    -4.629592   0.000028        0.000023
 1  -5.849858    -5.865107   0.000001        0.000001
 2  -4.882729    -5.208622   0.000013        0.000006
 3  -5.337053    -5.349304   0.000005        0.000004
 4  -5.325139    -5.372299   0.000005        0.000004
 ..       ...          ...        ...             ...
 77 -5.017593    -5.006342   0.000010        0.000010
 78 -5.578232    -5.576677   0.000003        0.000003
 79 -4.345823    -4.350099   0.000045        0.000045
 80 -5.901702    -5.675081   0.000001        0.000002
 81 -4.529737    -3.923937   0.000030        0.000119
 
 [82 rows x 4 columns])

In [18]:
aux=load_and_validate_model("param_20210913_055314_lgb_adjusted_r2",X,y)

------------------------ OPTIMAL PARAMETERS ------------------------
{'bagging_fraction': 0.9747766114833514, 'feature_fraction': 0.8946608757396138, 'learning_rate': 0.048335934814995346, 'max_depth': 11.685910466019436, 'min_child_weight': 13.355916840770721, 'min_split_gain': 0.00795711246669661, 'num_leaves': 108.61067784693134, 'num_iterations': 300, 'n_estimators': 1000}
-------------------------------------------------------------------
The r2 of prediction is: 0.9128162317124225
The MSE of prediction is: 0.048146653606830674
The RMSE of prediction is: 0.21942345728483698
The MAE of prediction is: 0.14351557692983558


In [20]:
df=aux[1]

In [21]:
df.head()

,y_true,y_predicted,y_true_10,y_predicted_10
0,-4.558148,-4.629592,0.000028,0.000023
1,-5.849858,-5.865107,0.000001,0.000001
2,-4.882729,-5.208622,0.000013,0.000006
3,-5.337053,-5.349304,0.000005,0.000004
4,-5.325139,-5.372299,0.000005,0.000004


In [23]:
df[df["y_true_10"]>0.00001]

,y_true,y_predicted,y_true_10,y_predicted_10
0,-4.558148,-4.629592,0.000028,0.000023
2,-4.882729,-5.208622,0.000013,0.000006
7,-4.982967,-5.233598,0.000010,0.000006
13,-4.387110,-4.200853,0.000041,0.000063
19,-4.407934,-4.515362,0.000039,0.000031
22,-4.201418,-4.352621,0.000063,0.000044
23,-4.775985,-4.733322,0.000017,0.000018
25,-3.254067,-3.726029,0.000557,0.000188
26,-2.528122,-3.528397,0.002964,0.000296
29,-4.119529,-4.018800,0.000076,0.000096


In [24]:
df[df["y_true_10"]>0.0001]

,y_true,y_predicted,y_true_10,y_predicted_10
25,-3.254067,-3.726029,0.000557,0.000188
26,-2.528122,-3.528397,0.002964,0.000296
35,-3.716021,-3.687190,0.000192,0.000205
43,-3.501965,-3.668473,0.000315,0.000215
50,-3.753994,-3.663359,0.000176,0.000217
60,-3.513428,-3.686435,0.000307,0.000206
63,-3.812197,-4.124028,0.000154,0.000075
70,-3.499215,-3.569816,0.000317,0.000269


In [25]:
aux=load_and_validate_model("param_20210913_020533_lgb_r2",X,y)
df=aux[1]

------------------------ OPTIMAL PARAMETERS ------------------------
{'bagging_fraction': 0.8693796326739299, 'feature_fraction': 0.8168553631386558, 'learning_rate': 0.0464041915708064, 'max_depth': 13.009239099967989, 'min_child_weight': 10.19191247052637, 'min_split_gain': 0.004793459623540061, 'num_leaves': 93.8608845306346, 'num_iterations': 300, 'n_estimators': 1000}
-------------------------------------------------------------------
The r2 of prediction is: 0.9065216981160615
The MSE of prediction is: 0.05162276773487457
The RMSE of prediction is: 0.22720644298715337
The MAE of prediction is: 0.15151940051952298


In [26]:
df[df["y_true_10"]>0.0001]

,y_true,y_predicted,y_true_10,y_predicted_10
25,-3.254067,-3.673389,0.000557,0.000212
26,-2.528122,-3.614213,0.002964,0.000243
35,-3.716021,-3.677709,0.000192,0.000210
43,-3.501965,-3.681524,0.000315,0.000208
50,-3.753994,-3.698844,0.000176,0.000200
60,-3.513428,-3.694179,0.000307,0.000202
63,-3.812197,-4.154395,0.000154,0.000070
70,-3.499215,-3.578884,0.000317,0.000264
